In [1]:
from distutils.command.config import config
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertConfig, get_linear_schedule_with_warmup
from tqdm import tqdm
import os
import time
from transformers import BertTokenizer
from transformers import logging
import processing
from sklearn import metrics
import warnings
import time
import sys
sys.path.append("D:/Experiment")
from MyKu import processing
logging.set_verbosity_error()

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)


cuda:0


In [2]:
train_data = processing.get_OLID_train_data()
test_data = processing.get_OLID_testA_data()

MAX_LEN = 60

def save_pretrained(model, path):
    # 保存模型，先利用os模块创建文件夹，后利用torch.save()写入模型文件
    os.makedirs(path, exist_ok=True)
    torch.save(model, os.path.join(path, 'model.pth'))

pretrained_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name, do_lower_case=True)

def preprocessing_for_bert(data):
    input_ids, attention_masks, labels = [], [], []
    for sent, label in data:
        encoded_sent = tokenizer.encode_plus(
            text=sent,
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            )
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))
        labels.append(label)
    input_ids = torch.Tensor(input_ids)
    attention_masks = torch.Tensor(attention_masks)
    # print(attention_masks)
    labels = torch.Tensor(labels)
    return input_ids, attention_masks, labels

train_inputs, train_masks, train_labels = preprocessing_for_bert(train_data)
test_inputs, test_masks, test_labels = preprocessing_for_bert(test_data)

train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
test_dataset = TensorDataset(test_inputs, test_masks, test_labels)
train_sampler = RandomSampler(train_dataset)
test_sampler = SequentialSampler(test_dataset)

train_iter = DataLoader(train_dataset, sampler=train_sampler, batch_size=32)
test_iter = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

In [3]:
def initialize_model(epoch=10):
    bert_model = MyBertModel(MAX_LEN, 2)
    bert_model.to(DEVICE)
    learning_rate = 2e-5
    optimizer = AdamW(
        bert_model.parameters(),
        lr=learning_rate,
        eps=1e-8
    )
    total_steps = len(train_iter) * epoch
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    return bert_model, optimizer, scheduler

loss_fn = nn.CrossEntropyLoss()

In [4]:
best_score = 0
def train(model, train_iter, test_iter, optimizer, scheduler, epochs=10, evaluation=None):
    for num_epoch in range(epochs):
        print(f"{'Epoch':^7} | {'每40个Batch':^9} | {'训练集 Loss':^12} | {'测试集 Loss':^10} | {'测试集准确率':^9} | {'时间':^9}")
        print("-" * 80)
        t0_epoch, t0_batch = time.time(), time.time()
        total_loss, batch_loss, batch_counts = 0, 0, 0
        model.train()
        for step, batch in enumerate(train_iter):
            batch_counts += 1
            b_input_ids, b_att_masks, b_labels = tuple(
                t.to(DEVICE, dtype=torch.int32) for t in batch)
            model.zero_grad()
            # b_input_ids = b_input_ids.to(dtype=torch.int32)
            output = model.classifer(model(b_input_ids, b_att_masks))
            # print(output.dtype)
            b_input_ids = b_input_ids.to(dtype=torch.int32)
            loss = loss_fn(output, b_labels.long())
            batch_loss += loss.item()
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            if(step % 40 == 0 and step != 0) or (step == len(train_iter) - 1):
                time_elapsed = time.time() - t0_batch
                print(
                    f"{num_epoch + 1:^7} | {step:^10}  | {batch_loss / batch_counts:^14.6f}  | {'-':^12} | {'-':^13} |  {time_elapsed:^9.2f}")
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        avg_train_loss = total_loss / len(train_iter)
        print('-' * 80)

        if evaluation:
            test_loss, test_accuracy, f1_score = evaluate(model, test_iter)
            time_elapsed = time.time() - t0_epoch
            print(f"{num_epoch + 1:^7} | {'-':^10} | {avg_train_loss:^14.6f} | {test_loss:^12.6f} | {test_accuracy:^12.2f} | {f1_score:^12.2f} | {time_elapsed:^9.2f}")
            print("-" * 80)
        print("\n")


def evaluate(model, test_dataloader):
    global best_score
    """
    在每个epoch后验证集上评估model性能
    """
    model.eval()
    # 准确率和误差
    test_accuracy = []
    test_loss = []
    pred_y, true_y = [], []
    # 验证集上的每个batch
    for batch in test_dataloader:
        # 放到GPU上
        b_input_ids, b_attn_masks, b_labels = tuple(
            t.to(DEVICE, dtype=torch.int32) for t in batch)
        # 计算结果，不计算梯度
        with torch.no_grad():
            # 放到model里面去跑，返回验证集的ouput就是一行三列的
            output = model.classifer(model(b_input_ids, b_attn_masks))
        # 计算误差
        loss = loss_fn(output, b_labels.long())
        test_loss.append(loss.item())
        pred = torch.argmax(output, dim=1).flatten()
        # get预测结果，这里就是求每行最大的索引咯，然后用flatten打平成一维
        # 计算准确率，这个就是俩比较，返回相同的个数, .cpu().numpy()就是把tensor从显卡上取出来然后转化为numpy类型的举证好用方法
        # 最后mean因为直接bool形了，也就是如果预测和label一样那就返回1，正好是正确的个数，求平均就是准确率了
        accuracy = (pred == b_labels).cpu().numpy().mean() * 100
        test_accuracy.append(accuracy)
        pred_y.extend(pred.tolist())
        true_y.extend(b_labels.tolist())
    # print(len(true_y))
    # 计算整体的平均正确率和loss
    print(metrics.confusion_matrix(true_y, pred_y))
    val_loss = np.mean(test_loss)
    val_accuracy = np.mean(test_accuracy)
    f1_socre = metrics.f1_score(true_y, pred_y, average="macro") * 100
    if f1_socre > 80 and f1_socre > best_score:
        best_score = f1_socre
        path = 'D:/Experiment_models_save/OLID/'
        save_pretrained(model, path)

    return val_loss, val_accuracy, f1_socre

In [5]:
class MyBertModel(nn.Module):

    def __init__(self, len, class_size, pretrained_name='bert-base-uncased'):
        super(MyBertModel, self).__init__()
        config = BertConfig.from_pretrained(pretrained_name)
        config.output_hidden_states = True
        # config.output_attentions = True
        config.return_dict = True
        self.bert = BertModel.from_pretrained(pretrained_name, config=config)
        self.w_h = nn.Parameter(torch.Tensor(len))
        self.b_n = nn.Parameter(torch.Tensor(768))
        self.W_h = nn.Parameter(torch.Tensor(768, 768))
        self.W_e = nn.Parameter(torch.Tensor(768, 768))
        self.W_hs = nn.Parameter(torch.Tensor(768, 768))
        self.W_L1 = nn.Parameter(torch.Tensor(768 * 3, 768))
        self.b_L1 = nn.Parameter(torch.Tensor(len, 768))
        self.W_L2 = nn.Parameter(torch.Tensor(768, class_size))
        self.b_L2 = nn.Parameter(torch.Tensor(len, class_size))
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.len = len
        self.softmax = nn.Softmax(dim=-1)
        self.classifer = nn.Linear(len, class_size)
        nn.init.uniform_(self.w_h, -0.1, 0.1)
        nn.init.uniform_(self.W_h, -0.1, 0.1)
        nn.init.uniform_(self.W_e, -0.1, 0.1)
        nn.init.uniform_(self.b_n, -0.1, 0.1)
        nn.init.uniform_(self.W_hs, -0.1, 0.1)
        nn.init.uniform_(self.W_L1, -0.1, 0.1)
        nn.init.uniform_(self.b_L1, -0.1, 0.1)
        nn.init.uniform_(self.W_L2, -0.1, 0.1)
        nn.init.uniform_(self.b_L2, -0.1, 0.1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state_cls = outputs[0] # Last_hidden size [32, 80, 768]
        Embedding = outputs[2][0] # Embedding size [32, 80, 768]
        hs = self.tanh(torch.matmul(Embedding.permute(0, 2, 1), self.w_h) + self.b_n) # hs size [32, 768]
        hs = torch.repeat_interleave(torch.unsqueeze(hs, 1), self.len, 1) #hs size [32, 80, 768]
        h_f = torch.cat((torch.cat((torch.matmul(last_hidden_state_cls, self.W_h), torch.matmul(Embedding, self.W_e)), dim=-1), torch.matmul(hs, self.W_hs)), dim=-1)
        # hf size [32, 80, 768 * 3]
        L1 = self.relu(torch.matmul(h_f, self.W_L1) + self.b_L1) # L1 size [32, 80, 768]
        L2 = self.softmax(torch.matmul(L1, self.W_L2) + self.b_L2) # L2 size [32, 80, 2]
        outs = L2[:, :, 1]
        return outs
    

In [6]:
for i in range(10):
      bert_model, optimizer, scheduler = initialize_model(2)
      print("Start training and testing:\n")
      train(bert_model, train_iter,
            test_iter, optimizer, scheduler, epochs=3, evaluation=True)

Start training and testing:

 Epoch  | 每40个Batch |   训练集 Loss   |  测试集 Loss  |  测试集准确率   |    时间    
--------------------------------------------------------------------------------
   1    |     40      |    0.653406     |      -       |       -       |    15.20  
   1    |     80      |    0.638056     |      -       |       -       |    13.74  
   1    |    120      |    0.588376     |      -       |       -       |    13.78  
   1    |    160      |    0.535470     |      -       |       -       |    13.85  


KeyboardInterrupt: 

In [ ]:
for i in range(10):
      bert_model, optimizer, scheduler = initialize_model(2)
      print("Start training and testing:\n")
      train(bert_model, train_iter,
            test_iter, optimizer, scheduler, epochs=3, evaluation=True)

Start training and testing:

 Epoch  | 每40个Batch |   训练集 Loss   |  测试集 Loss  |  测试集准确率   |    时间    
--------------------------------------------------------------------------------
   1    |     40      |    0.653406     |      -       |       -       |    15.20  
   1    |     80      |    0.638056     |      -       |       -       |    13.74  
   1    |    120      |    0.588376     |      -       |       -       |    13.78  
   1    |    160      |    0.535470     |      -       |       -       |    13.85  


KeyboardInterrupt: 